# Collecting the dataset

In [1]:
import os 
import urllib
import urllib.request 
from zipfile import ZipFile
import numpy as np
import cv2


URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'FashionMNISTImage'

if not os.path.isfile(FILE): #Checks to see if the file already exists 
    urllib.request.urlretrieve(URL, FILE) #Download the image 

if not os.path.isdir(FOLDER):
    with ZipFile(FILE) as zippedImages:
        zippedImages.extractall(FOLDER) #unzips the folder 

def loadData(dataset, path):
    #path refers to the folder
    #path refers to the batches of data 
    labels = os.path.join(path, dataset)

    X = []
    y = []
    for folderClass in os.listdir(labels):
        #folderClass contains the class classification
        for images in os.listdir(os.path.join(path, dataset, folderClass)):
            readImage = cv2.imread(os.path.join(path, dataset, folderClass, images), cv2.IMREAD_UNCHANGED)
            #IMREAD Makes it read a grey scale from -1 to 1 (The image data has no colours)
            X.append(readImage)
            y.append(folderClass)
    return np.array(X), np.array(y, dtype = np.uint8)

def createData(path):
    X,y = loadData('train', path)
    Xtest, ytest = loadData('test', path)

    return X, y, Xtest, ytest

X, y, Xtest, ytest = createData(FOLDER)


keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)

X = X[keys]
y = y[keys]



X = (X.reshape(X.shape[0], -1).astype(np.float32) - 127.5) /127.5
Xtest = (Xtest.reshape(Xtest.shape[0], -1).astype('float32') - 127.5) /127.5

print(Xtest.shape)
print(X.shape)


(10000, 784)
(60000, 784)


## Training the neural network

In [2]:

import Neural_network_code.neuroNetwork as nn
import Neural_network_code.Optimizers as op 

#The reshaping exists to turn the cv2 conversion, which presents itself in rows of rows into a single row and the normalizing is\
#to make values between -1 and 0 for which neuralnetworks work best with 

model = nn.NeuroModel()

model.add(nn.neuroNetworkLayer(X.shape[1], 128))
model.add(nn.activateRELU())
model.add(nn.neuroNetworkLayer(128, 128))
model.add(nn.activateRELU())
model.add(nn.neuroNetworkLayer(128, 10))
model.add(nn.softMaxFunction())
model.set(nn.LossCategoricalCrossEntropy(), op.Optimizer_Adam(rate = 0.001, decay = 1e-3), nn.categoricalAccuracy())
model.finalize()

model.train(X, y, 10, 1,  batchSize = 128)
model.evaluate(Xtest, ytest, batchSize = 128)




s 0
p 0.078125
l 2.302454710006714
nl 2.3024547
rl 0.0


s 100
p 0.8203125
l 0.5567699074745178
nl 0.5567699
rl 0.0


s 200
p 0.71875
l 0.7896583080291748
nl 0.7896583
rl 0.0


s 300
p 0.8828125
l 0.43492525815963745
nl 0.43492526
rl 0.0


s 400
p 0.8125
l 0.4815658628940582
nl 0.48156586
rl 0.0


Validation
e 0
p 0.7594651442307693
l 0.6522679511680562
nl 0.6522679511680562
rl 0.0


s 0
p 0.8671875
l 0.41483283042907715
nl 0.41483283
rl 0.0


s 100
p 0.859375
l 0.4567376375198364
nl 0.45673764
rl 0.0


s 200
p 0.8203125
l 0.5906740427017212
nl 0.59067404
rl 0.0


s 300
p 0.859375
l 0.37812677025794983
nl 0.37812677
rl 0.0


s 400
p 0.8515625
l 0.4285283088684082
nl 0.4285283
rl 0.0


Validation
e 1
p 0.8422309027777778
l 0.43756253170406717
nl 0.43756253170406717
rl 0.0


s 0
p 0.859375
l 0.3867972493171692
nl 0.38679725
rl 0.0


s 100
p 0.859375
l 0.3857634365558624
nl 0.38576344
rl 0.0


s 200
p 0.828125
l 0.5429725646972656
nl 0.54297256
rl 0.0


s 300
p 0.875
l 0.36510753631591797

## Saving the network

In [3]:
import pickle 

with open('NeuroNetworkModel.pickle', 'wb') as file:
    pickle.dump(model, file)
